In [1]:
# All Imports
import sys 
import numpy as np
import json
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
#from sklearn.cluster import DBSCAN
import collections
#from sklearn.neighbors import NearestNeighbors
#from gensim.test.utils import common_texts
import pickle
from sklearn.decomposition import PCA
#from gensim.models.doc2vec import Doc2Vec, TaggedDocument,KeyedVectors 
#from gensim.test.utils import get_tmpfile
#import nltk
#from nltk.sentiment import SentimentIntensityAnalyzer
#from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

#from sklearn.cluster import KMeans

In [2]:
with open('tracks_train.json','r') as file:
    train_docs = json.load(file) 
print(len(train_docs))
with open('tracks_test.json','r') as file:
    test_docs = json.load(file) 
print(len(test_docs))

filtereddocs = pickle.load( open( "counted_bows", "rb" ) ) #train_documents
print(len(filtereddocs))

210519
27143
123984


In [9]:
test_documents_json = pickle.load( open( "counted_bows_test", "rb" ) )

In [4]:
all_w_k = []
for i in bow_vectors.values():    
    all_w_k += [int(x) for x in i.keys()]
set_wk = set(all_w_k)
print(len(set_wk))

4126


In [91]:
print(set_wk[-1])

TypeError: 'set' object is not subscriptable

In [5]:
def dict_to_vector(words):
    vector = []
    for i in list(set_wk):
        if str(i) in words:
            vector.append(int(words[str(i)]))
        else:
            vector.append(0)
    return vector

bow_vectors = {}
for doc in filtereddocs:
    did = doc[1][0]
    if did in train_docs:
        bow_vectors[did] = dict_to_vector(train_docs[did]['words'])
        #bow_vectors[did] = train_docs[did]['words']
    if did in test_docs:
        bow_vectors[did] = dict_to_vector(test_docs[did]['words'])
        #bow_vectors[did] = test_docs[did]['words']
#print(len(bow_vectors))
#print(bow_vectors['4252601'])
#np_vectors = np.array(list(bow_vectors.values()))

In [2]:
np_vectors = pickle.load( open( "bow_vectors1.p", "rb" ) )
#np_vectors  = np.array(list(bow_vectors.values()))

word_counts = np.sum(np_vectors, 0)
to_delete = [i for i in range(len(word_counts)) if word_counts[i] > 180000 or word_counts[i] < 30]

print(len(to_delete), len(word_counts))
#np_vectors = np.delete(np_vectors, to_delete, 0)
print(np.shape(np_vectors) )
np_vectors_small = np.delete(np_vectors, to_delete, 1)
print(np.shape(np_vectors_small) )

536 4126
(123984, 4126)


In [4]:
#np_vectors = np.array(list(bow_vectors.values()))
with open("bow_vectors1.p","wb") as inf_json:
    pickle.dump(np_vectors_small,inf_json)

In [6]:
print(len(princomps),princomps[0])

123984 [  5.60082012   1.2115934   -1.58404199  -1.28377742   2.78026157
   2.26953982   2.81396657  -8.97400708   1.75882056   2.50610376
   0.10814506  -3.23075919   1.01806087  -0.03298465   0.87926803
  -0.50588243  -1.20978913  -0.89778248   2.29363993  -3.60679287
  -0.93045357  -1.82296988  -1.30808822  -2.16945148   1.91844061
   0.4202481   -0.79957688   2.7171709    0.63653754  -0.96881233
  -1.29314285  -0.1621938    2.26032537  -0.02515892   1.18822424
  -0.13719202   1.69959856   0.50883139   0.85513438   1.16878393
  -0.68376373   0.30575952  -0.34644185   1.95321251   1.98519774
  -0.98183511  -1.325814    -0.49732792   1.92799672  -1.94114801
   3.17586201  -1.77407584  -1.30839319  -2.89797323  -4.22619108
   4.57983388   5.5679175    3.04729063   0.43123507   0.83097138
   4.69073922   2.16673812   2.26284358  -4.11616135  -2.47295079
  -3.45589556   1.93132825  -2.82642259  -0.54406746  13.92021807
   4.08376568   1.81762692 -10.49030678  -6.2536038   -1.35011015
   

In [41]:
#np_vectors = pickle.load( open( "bow_vectors1.p", "rb" ) )

pca = PCA(n_components=20)
princomps = pca.fit_transform(np_vectors)
#X_embedded = TSNE(n_components=2, perplexity=100, learning_rate=600).fit_transform(princomps)
with open("bow_vectors2.p","wb") as inf_json:
    pickle.dump(princomps,inf_json)

In [65]:
def paint_test_genres():
    test_ids = [i[1] for i in test_documents_json]
    inf_vecs = pickle.load( open( "inferred_vectors_dict.p", "rb" ) ) #test_documents
    test_vecs = [(inf_vecs[vec[0]],vec[1]) for vec in test_ids if vec[0] in bow_vectors]
    X = np.array([i[0] for i in test_vecs])

    genres = [i[1] for i in test_vecs]
    counter = collections.Counter(genres)
    for i in range(len(genres)):
        if counter[genres[i]] < 55:
            genres[i] = 0
    unique_colors = sorted(list(set(genres)),key= lambda x: counter[x])
    colors = [unique_colors.index(i) for i in genres]

    X_embedded = TSNE(n_components=2, perplexity=50).fit_transform(X)

    with open('genres.json','r') as file2:
        genre_names = json.load(file2) 
    genre_mapping = {i: [x['name'] for x in genre_names if x['genre_id'] == i] for i in unique_colors}

    plt.figure(figsize=(25, 25))

    plt.scatter([i[0] for i in X_embedded],[i[1] for i in X_embedded],c=colors,cmap='tab20', s=20)

    plt.scatter([-60 for i in list(set(colors))],[(i+30)*2 for i in list(set(colors))],c=list(set(colors)), cmap='tab20', s=20)
    for i in range(len(unique_colors)):
        txt = genre_mapping[unique_colors[i]]
        plt.annotate(txt, (-59, (i+30)*2))

In [55]:
#print(bow_vectors[0])

def paint_test_genres_bow():
    test_ids = [i[1] for i in test_documents_json ]
    test_vecs = [(bow_vectors[vec[0]],vec[1]) for vec in test_ids if vec[0] in bow_vectors]
    X = np.array([i[0] for i in test_vecs])
    #print(X[0])

    genres = [i[1] for i in test_vecs]
    counter = collections.Counter(genres)
    for i in range(len(genres)):
        if counter[genres[i]] < 55:
            genres[i] = 0
    unique_colors = sorted(list(set(genres)),key= lambda x: counter[x])
    colors = [unique_colors.index(i) for i in genres]

    X_embedded = TSNE(n_components=2, perplexity=50).fit_transform(X)

    with open('genres.json','r') as file2:
        genre_names = json.load(file2) 
    genre_mapping = {i: [x['name'] for x in genre_names if x['genre_id'] == i] for i in unique_colors}

    plt.figure(figsize=(25, 25))

    plt.scatter([i[0] for i in X_embedded],[i[1] for i in X_embedded],c=colors,cmap='tab20', s=20)
    #for i in range(len(X_embedded)):
    #    plt.annotate(test_ids[i][0], (X_embedded[i][0], X_embedded[i][1]))
    plt.scatter([-40 for i in list(set(colors))],[(i) for i in list(set(colors))],c=list(set(colors)), cmap='tab20', s=20)
    for i in range(len(unique_colors)):
        txt = genre_mapping[unique_colors[i]]
        plt.annotate(txt, (-39, (i)))

In [ ]:
#X_emb = TSNE(n_components=2, perplexity=50, learning_rate=500).fit_transform(princomps[5000:15000])
#df = pd.DataFrame(np_vectors[:10000,300:360])
#pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(25,25))

import seaborn as sn

covMatrix = np.cov(np_vectors[:5000,300:400],bias=True)
sn.heatmap(covMatrix, annot=True, fmt='g')
plt.show()

#plt.figure(figsize=(25, 25))
#plt.scatter([i[6] for i in princomps],[i[7] for i in princomps],c=[i[8] for i in princomps],cmap='gist_rainbow', s=15)